# Generatory, Yield

In [1]:
mylist = [0, 1, 4] #tworzenie listy
for i in mylist:
    print(i)

0
1
4


In [2]:
#wzór na listę, list comprehension, lista, która dostała generator do konstruktora
mylist = [x*x for x in range(3)] 
for i in mylist:
    print(i)

0
1
4


In [3]:
#wzór na generator, potencjalną możliwość utworzenia listy, zmniejsza obciążenie komputera i przyspiesza działanie
#generator stworzony przy pomocy wyrażenia generatorowego
mylist = (x*x for x in range(3))  
for i in mylist:
    print(i)

0
1
4


In [12]:
#generator stworzony przy pomocy funkcji
#zakres może być zmienny
#normalną funkcję kończymy słówkiem return i wartością zwracaną
#w przypadku generatorów zamiast return używamy słówka kluczowego yield
#yield zwraca wartość
#zwykła funkcja z return się wykona od początku
#w przypadku yield wracamy na miejsce, w którym wyszliśmy, funkcja dodatkowo zapamiętuje swój stan od poprzedniego wyjścia

def createGenerator(n):
    mylist = range(n)  
    for i in mylist:
        yield i*i #natychmiastowe zwrócenie wartości po otrzymaniu i
        
for i in createGenerator(3):
    print(i) #nie trafiamy na sam początek createGenerator, tylko tam, gdzie poprzednio wyszliśmy; tutaj wykonujemy kolejny krok pętli

0
1
4


In [13]:
generator = createGenerator(3)

In [14]:
for i in generator:
    print(i)

0
1
4


In [15]:
for i in generator: #generatory są jednorazowe
    print(i)

# Fail2Ban

Jeżeli z jednego adresu IP jest za dużo żądań, wrzucamy do kwarantanny.
Ustawiamy ile jest możliwych uderzeń w trakcie jednego czasu.
As you can see in my example, I have set up 300 maxretry and 300 for findtime, so, we need to have 300 GETs from the same IP in a time window of 300 seconds to have the originating IP blocked.

# Scraper

Scrapy doesn’t wait a fixed amount of time between requests, but uses a random interval between 0.5 * DOWNLOAD_DELAY and 1.5 * DOWNLOAD_DELAY.

In [16]:
import scrapy
import scrapy.crawler as crawler
from bs4 import BeautifulSoup

from scrapy.crawler import CrawlerProcess

class MySpider(scrapy.Spider):
    name = 'myspider'
    start_urls = [
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1'
        ]
   
    custom_settings = {
        'DOWNLOAD_DELAY': '4.0', #obchodzimy zabezpieczenia Fail2Ban, ustawiając przerwę
    }

    top_url = 'https://www.gumtree.pl'
    def parse(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        soup = BeautifulSoup(response.body, 'lxml')
        link_tabs = soup.findAll("div", {"class": "result-link"}) #szukamy result link, czyli ramki ze screena 1
        item_urls = []
        for tab in link_tabs:
            hrefs = tab.findAll("a", {"class": "href-link"}) #szukamy parametru o atrybucie href link, screen 2
            for h in hrefs:
                item_urls.append(self.top_url + h["href"]) #dopisujemy 'https://www.gumtree.pl', bo otrzymany adres jest względny, przygotowujemy listę wszystkich linków
                
        
        for item_url in item_urls:
            yield scrapy.Request(item_url, self.parse_item) #item_url - odwiedzanie strony, #self.parse_item - przetworzenie przy pomocy funkcji

    def parse_item(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))


In [17]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
process.crawl(MySpider)
process.start()

2019-04-04 19:50:03 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-04-04 19:50:03 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 17.9.0, Python 3.6.7 (default, Oct 22 2018, 11:32:17) - [GCC 8.2.0], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.4.0-17134-Microsoft-x86_64-with-Ubuntu-18.04-bionic
2019-04-04 19:50:03 [scrapy.crawler] INFO: Overridden settings: {'DOWNLOAD_DELAY': '4.0', 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-04-04 19:50:03 [scrapy.extensions.telnet] INFO: Telnet Password: 84eeacb84f5c1344
2019-04-04 19:50:04 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-04-04 19:50:06 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloade

2019-04-04 19:51:06 [scrapy.extensions.logstats] INFO: Crawled 12 pages (at 12 pages/min), scraped 0 items (at 0 items/min)
2019-04-04 19:51:07 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/srodmiescie-5-pokoi-rondo-onz/1004684896470911158184709> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-04 19:51:07 [myspider] INFO: Got successful response from https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/srodmiescie/srodmiescie-5-pokoi-rondo-onz/1004684896470911158184709
2019-04-04 19:51:12 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/wawer/nowoczesne-osiedle-na-wawrze/1004685156880911543290009> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-04 19:51:12 [myspider] INFO: Got successful response from https://www.gumtree.pl/a-mies

In [1]:
#pająk z paginacją, czyli chodzący po stronach

import scrapy
import scrapy.crawler as crawler
from bs4 import BeautifulSoup

from scrapy.crawler import CrawlerProcess

class MyPaginatingSpider(scrapy.Spider):
    name = 'mypaginatingspider'
    start_urls = [
        'https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1'
        ]
   
    custom_settings = {
        'DOWNLOAD_DELAY': '6.0',
    }

    top_url = 'https://www.gumtree.pl'
    def parse(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))
        soup = BeautifulSoup(response.body, 'lxml')
        link_tabs = soup.findAll("div", {"class": "result-link"})
        item_urls = []
        next_urls = []
        for tab in link_tabs:
            hrefs = tab.findAll("a", {"class": "href-link"})
            for h in hrefs:
                item_urls.append(self.top_url + h["href"])
            
        nexts = soup.findAll("a", {"class": "next"})
        
        for h in nexts:
            for h in hrefs:
                next_urls.append(self.top_url + h["href"])
        
        print (next_urls)
            
        for item_url in item_urls:
            yield scrapy.Request(item_url, self.parse_item)

        for next_url in next_urls: #odwiedzanie następnych stron z linkami
            yield scrapy.Request(item_url, self.parse)
    
            
    def parse_item(self, response):
        self.logger.info('Got successful response from {}'.format(response.url))



In [2]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})
process.crawl(MyPaginatingSpider)
process.start()

2019-04-04 20:08:39 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-04-04 20:08:39 [scrapy.utils.log] INFO: Versions: lxml 4.3.2.0, libxml2 2.9.9, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 17.9.0, Python 3.6.7 (default, Oct 22 2018, 11:32:17) - [GCC 8.2.0], pyOpenSSL 17.5.0 (OpenSSL 1.1.0g  2 Nov 2017), cryptography 2.1.4, Platform Linux-4.4.0-17134-Microsoft-x86_64-with-Ubuntu-18.04-bionic
2019-04-04 20:08:39 [scrapy.crawler] INFO: Overridden settings: {'DOWNLOAD_DELAY': '6.0', 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-04-04 20:08:39 [scrapy.extensions.telnet] INFO: Telnet Password: e248f01720ddaab4
2019-04-04 20:08:39 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-04-04 20:08:39 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloade

['https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bielany/mieszkanie-warszawa-bielany-90m2-nr-76578-3645-oms/1004685300410910535030709']


2019-04-04 20:08:45 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/bezposrednio-33-5-m2-na-starym-mokotowie-+-ul-madalinskiego/1004346805610912096532209> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-04 20:08:46 [mypaginatingspider] INFO: Got successful response from https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/mokotow/bezposrednio-33-5-m2-na-starym-mokotowie-+-ul-madalinskiego/1004346805610912096532209
2019-04-04 20:08:54 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bielany/mieszkanie-warszawa-bielany-90m2-nr-76578-3645-oms/1004685300410910535030709> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-04 20:08:54 [mypaginatingspider] INFO: Got successful response from https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kup

2019-04-04 20:10:39 [scrapy.extensions.logstats] INFO: Crawled 16 pages (at 7 pages/min), scraped 0 items (at 0 items/min)
2019-04-04 20:10:41 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/piaseczno/mieszkanie-chylice-190-22m2-nr-bs5+ms+224919+107/1004685300700911543290009> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-04 20:10:42 [mypaginatingspider] INFO: Got successful response from https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/piaseczno/mieszkanie-chylice-190-22m2-nr-bs5+ms+224919+107/1004685300700911543290009
2019-04-04 20:10:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://www.gumtree.pl/a-mieszkania-i-domy-sprzedam-i-kupie/bialoleka/nowe-mieszkanie-2-pok-%2B-taras-+-bialoleka-2019/1004685298690911379840509> (referer: https://www.gumtree.pl/s-mieszkania-i-domy-sprzedam-i-kupie/mazowieckie/v1c9073l3200001p1)
2019-04-04 20:10:48 [mypagin

In [ ]:
michal.korzycki@se.com